In [0]:
%run "/Workspace/Users/dbxdeschrtrial2@gmail.com/databricks/Other/common"


# read bonze roads

In [0]:
dbutils.widgets.text(name="env",defaultValue="",label=" Enter the environment in lower case")
global env
env = dbutils.widgets.get("env")

In [0]:

def read_BronzeRoadsTable(environment):
    print('Reading the Bronze Table raw_roads Data : ',end='')
    df_bronzeRoads = (spark.readStream
                    .table(f"`{environment}_catalog`.`bronze`.raw_roads")
                    )
    print(f'Reading {environment}_catalog.bronze.raw_roads Success!')
    print("**********************************")
    return df_bronzeRoads

# Transformation create columns

## Creating road_category_name column

In [0]:
def road_Category(df):
    print('Creating Road Category Name Column: ', end='')
    from pyspark.sql.functions import when,col

    df_road_Cat = df.withColumn("Road_Category_Name",
                  when(col('Road_Category') == 'TA', 'Class A Trunk Road')
                  .when(col('Road_Category') == 'TM', 'Class A Trunk Motor')
                   .when(col('Road_Category') == 'PA','Class A Principal road')
                    .when(col('Road_Category') == 'PM','Class A Principal Motorway')
                    .when(col('Road_Category') == 'M','Class B road')
                    .otherwise('NA')
                  
                  )
    print('Success!! ')
    print('***********************')
    return df_road_Cat

## create roadCategory

In [0]:
def roadCategory(df):
    print('create road category name: ',end='')
    from pyspark.sql.functions import col, when
    
    df_road_Cat = df.withColumn('Road_Category', 
                       when(col('Road_Category') == 'TA', 'Class A Truck Road')
                       .when(col('Road_Category') == 'TM', 'Class A Truck Motor')
                       .when(col('Road_Category') == 'PA', 'Class A Principal Motorway')
                       .when(col('Road_Category') == 'M', 'Class B Road')
                       .otherwise('NA')
                      )
                      
    print('create road category name completed'
    )

## create road_type

In [0]:
def roadtype(df):
    print('create road category name: ',end='')
    from pyspark.sql.functions import col, when
    
    df_road_Cat = df.withColumn('Road_Category', 
                       when(col('Road_Category_name').like('%Class A%'), 'Major')
                       .when(col('Road_Category_name').like('%Class B%'), 'Minor')
                       .otherwise('NA')
                      )
                      
    print('create road type completed')

In [0]:
def handle_NULLs2(df,columns):
    print('Replacing NULL values on String Columns with "Unknown" ' , end='')
    df_string = df.fillna('Unknown',subset= columns)
    print('Successs!! ')

    print('Replacing NULL values on Numeric Columns with "0" ' , end='')
    df_clean = df_string.fillna(0,subset = columns)
    print('*******************************************************')
    print('Success!! ')

    return df_clean

## Road_type

In [0]:
def road_Type(df):
    print('Creating Road Type Name Column: ', end='')
    from pyspark.sql.functions import when,col

    df_road_Type = df.withColumn("Road_Type",
                  when(col('Road_Category_Name').like('%Class A%'),'Major')
                  .when(col('Road_Category_Name').like('%Class B%'),'Minor')
                    .otherwise('NA')
                  
                  )
    print('Success!! ')
    print('***********************')
    return df_road_Type

# WRITE to silver.Silver_roads

In [0]:
def write_Roads_SilverTable(StreamingDF,environment):
    print('Writing the silver_roads Data : ',end='') 

    write_StreamSilver_R = (StreamingDF.writeStream
                .format('delta')
                .option("checkpointLocation",f'{checkpoint_path}/SilverRoadsLoad/raw_roads/') 
                .outputMode('append')
                .queryName("SilverRoadsWriteStream")
                .trigger(availableNow=True)
                .toTable(f"`{environment}_catalog`.`silver`.`silver_roads`"))
    
    write_StreamSilver_R.awaitTermination()
    print(f'Writing `{environment}_catalog`.`silver`.`silver_roads` Success!')

# Main

In [0]:
# Initiate variable path
Initglobalvarpath(env)

# read Bronze Table
df_roads = read_BronzeRoadsTable(env)

# Transformation silver layer
# Cleaning Data
df_noDups = remove_Dups(df_roads)
AllColumns = df_noDups.schema.names
df_clean = handle_NULLs(df_noDups,AllColumns)

# Transformation silver layer
## Creating Road_Category_name 
df_roadCat = road_Category(df_clean)
## Creating Road_Type column
df_type = road_Type(df_roadCat)

## Writing data to silver_roads table
write_Roads_SilverTable(df_type,env)

# query the silver table

In [0]:
%sql
select * from dev_catalog.silver.silver_roads 
where Road_Category_Name !='NA'
limit 100